# Pilot4a -- Compute the baseline decodability of rule representations (Logic, Sensory, and Motor rules)

## Use MVPA classifications to decode

## Takuya Ito
#### 02/26/2019

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import multiprocessing as mp
import scipy.stats as stats
import nibabel as nib
import os
os.environ['OMP_NUM_THREADS'] = str(1)
import statsmodels.api as sm
import sklearn.svm as svm
import statsmodels.sandbox.stats.multicomp as mc
import sklearn
from sklearn.feature_selection import f_classif
import seaborn as sns
import h5py
os.sys.path.append('glmScripts/')
import taskGLMPipeline as tgp
import statsmodels.api as sm
import tools

sns.set_style("whitegrid")
plt.rcParams["font.family"] = "FreeSans"


/home/tito/miniconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Excluding 084
subjNums = ['013','014','016','017','018','021','023','024','026','027','028','030','031','032','033',
            '034','035','037','038','039','040','041','042','043','045','046','047','048','049','050',
            '053','055','056','057','058','062','063','066','067','068','069','070','072','074','075',
            '076','077','081','085','086','087','088','090','092','093','094','095','097','098','099',
            '101','102','103','104','105','106','108','109','110','111','112','114','115','117','119',
            '120','121','122','123','124','125','126','127','128','129','130','131','132','134','135',
            '136','137','138','139','140','141']



basedir = '/projects3/SRActFlow/'

# Using final partition
networkdef = np.loadtxt('/projects3/NetworkDiversity/data/network_partition.txt')
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}
networks = networkmappings.keys()

xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net

## General parameters/variables
nParcels = 360
nSubjs = len(subjNums)

glasserfile2 = '/projects/AnalysisTools/ParcelsGlasser2016/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile2).get_data()
glasser2 = np.squeeze(glasser2)

sortednets = np.sort(xticks.keys())
orderednetworks = []
for net in sortednets: orderednetworks.append(xticks[net])
    
networkpalette = ['royalblue','slateblue','paleturquoise','darkorchid','limegreen',
                  'lightseagreen','yellow','orchid','r','peru','orange','olivedrab']
networkpalette = np.asarray(networkpalette)

OrderedNetworks = ['VIS1','VIS2','SMN','CON','DAN','LAN','FPN','AUD','DMN','PMM','VMM','ORA']

# 1.0 Define functions for motor response decodings

In [33]:
from scipy.spatial.distance import cdist

def ruleDecoding(data, rois=np.arange(1,nParcels+1), ncvs=100, nproc=5):
    """
    Run an across-subject classification
    Decode input stimuli (4-way decoding) for each sensory rule type
    """
        
    nstim = data_task.shape[1] # Number of stimuli per stimulus-type
    
    nSubjs = data.shape[2]

    netStats = np.zeros((len(rois),nstim*nSubjs))


    nsamples = nSubjs * nstim

    # Label array for supervised learning
    labels = np.tile(range(nstim),nSubjs)
    subjarray = np.repeat(range(nSubjs),nstim)

    # Run SVM classifications on network-level activation patterns across subjects
    roicount = 0
    for roi in rois:
#         print 'Computing rule decoding for ROI', roicount, '/', len(rois)
        roi_ind = np.where(glasser2==roi)[0]
        nfeatures = len(roi_ind)
        roi_ind.shape = (len(roi_ind),1)       

        svm_mat = np.zeros((nsamples,roi_ind.shape[0]))
        samplecount = 0
        scount = 0
        for subj in range(len(subjNums)):
            roidata = np.squeeze(data[roi_ind,:,scount])
            svm_mat[samplecount:(samplecount+nstim),:] = roidata.T

            scount += 1
            samplecount += nstim

            # Spatially demean matrix across features
            samplemean = np.mean(svm_mat,axis=1)
            samplemean.shape = (len(samplemean),1)
            svm_mat = svm_mat - samplemean

        scores = randomSplitLOOBaselineCV(ncvs, svm_mat, labels, subjarray, nproc=nproc)
        netStats[roicount,:] = scores
        roicount += 1


    return netStats

def randomSplitLOOBaselineCV(ncvs, svm_mat, labels, subjarray, nproc=5):
    """
    Runs cross validation for an across-subject SVM analysis
    """
    
    ntasks = len(np.unique(labels))
    nsamples = svm_mat.shape[0]
    nsubjs = nsamples/ntasks

    subjects = np.unique(subjarray)
    indices = np.arange(nsamples)
    
    numsubjs_perfold = 1
    if nsubjs%numsubjs_perfold!=0: 
        raise Exception("Error: Folds don't match number of subjects")
        
    nfolds = nsubjs/numsubjs_perfold
    subj_array_folds = subjarray.copy()
    
    inputs = [] 
    
    for fold in range(nfolds):
        test_subjs = np.random.choice(subj_array_folds,numsubjs_perfold,replace=False)
        train_subjs_all = np.delete(subjects,test_subjs)
        for cv in range(ncvs):
            # Randomly sample half of train set subjects for each cv (CV bootstrapping)
            train_subjs = np.random.choice(train_subjs_all,
                                         int(np.floor(len(train_subjs_all)*(4.0))),
                                         replace=True)

            train_ind = []
            for subj in train_subjs:
                train_ind.extend(np.where(subjarray==subj)[0])

            test_ind = []
            for subj in test_subjs:
                test_ind.extend(np.where(subjarray==subj)[0])
            
            train_ind = np.asarray(train_ind)
            test_ind = np.asarray(test_ind)

            trainset = svm_mat[train_ind,:]
            testset = svm_mat[test_ind,:]

            # Normalize trainset and testset
            mean = np.mean(svm_mat[train_ind,:],axis=0)
            mean.shape = (1,len(mean))
            std = np.std(svm_mat[train_ind,:],axis=0)
            std.shape = (1,len(std))

            trainset = np.divide((trainset - mean),std)
            testset = np.divide((testset - mean),std)
            
            trainlabels = labels[train_ind]
            testlabels = labels[test_ind]

#             ## Feature selection and downsampling
#             unique_labels = np.unique(labels)
#             feat1_labs = np.where(trainlabels==unique_labels[0])[0]
#             feat2_labs = np.where(trainlabels==unique_labels[1])[0]
#             # Perform t-test
#             t, p = stats.ttest_rel(trainset[feat1_labs,:],trainset[feat2_labs,:],axis=0)
#             h0, qs = mc.fdrcorrection0(p)
# #             h0 = p<0.1
#             # Construct feature masks
#             feat1_mask = np.multiply(t>0,h0).astype(bool)
#             feat2_mask = np.multiply(t<0,h0).astype(bool)
# #             feat1_mask = t>0
# #             feat2_mask = t<0
#             # Downsample training set into original vertices into 2 ROI signals
#             trainset_downsampled = np.zeros((trainset.shape[0],2))
#             trainset_downsampled[:,0] = np.nanmean(trainset[:,feat1_mask],axis=1)
#             trainset_downsampled[:,1] = np.nanmean(trainset[:,feat2_mask],axis=1)
#             trainset_downsampled = trainset[:,h0]
#             # Downsample test set into original vertices
#             testset_downsampled = np.zeros((testset.shape[0],2))
#             testset_downsampled[:,0] = np.nanmean(testset[:,feat1_mask],axis=1)
#             testset_downsampled[:,1] = np.nanmean(testset[:,feat2_mask],axis=1)
#             testset_downsampled = testset[:,h0]
            
#             if np.sum(feat1_mask)==0 or np.sum(feat2_mask==0):
#                 inputs.append((trainset,testset,trainlabels,testlabels))
#             else:
#                 inputs.append((trainset_downsampled,testset_downsampled,trainlabels,testlabels))
                
            inputs.append((trainset,testset,trainlabels,testlabels))
            
        
        subj_array_folds = np.delete(subj_array_folds,test_subjs)
        
    pool = mp.Pool(processes=nproc)
    scores = pool.map_async(_decoding,inputs).get()
    pool.close()
    pool.join()
    
#     subj_acc = np.zeros((len(subjects),))
#     scount = 0
#     i = 0
#     for subj in subjects:
#         subjmean = []
#         for cv in range(ncvs):
#             subjmean.append(scores[i])
#             i += 1
        
#         subj_acc[scount] = np.mean(subjmean)
        
#         scount += 1

#     return subj_acc
    acc = []
    for score in scores:
        acc.extend(score)
    return acc

def _decoding((trainset,testset,trainlabels,testlabels)):

# #     clf = sklearn.linear_model.LogisticRegression()
#     clf = svm.SVC(C=1.0, kernel='linear')

#     clf.fit(trainset,trainlabels)
#     predictions = clf.predict(testset)
#     acc = predictions==testlabels
#     score = np.mean(acc)

    unique_cond = np.unique(trainlabels)
    rdm = np.zeros((len(unique_cond),len(unique_cond)))
    acc = []
    for cond1 in unique_cond:
        mismatches = []
        prototype_ind = np.where(trainlabels==cond1)[0]
        prototype = np.mean(trainset[prototype_ind,:],axis=0)
        for cond2 in unique_cond:
            test_ind = np.where(testlabels==cond2)[0]
            test = np.mean(testset[test_ind,:],axis=0)
            if cond1 == cond2: 
                correct = stats.spearmanr(prototype,test)[0]
            else:
                mismatches.append(stats.spearmanr(prototype,test)[0])
        
        if correct > np.max(mismatches): 
            acc.append(1.0)
        else:
            acc.append(0.0)
    
    return acc

## 1.01 Load data for LOGIC rules

In [34]:
tools = reload(tools)

In [35]:
# gsr = True
n_rules = 4
data_task = np.zeros((len(glasser2),n_rules,len(subjNums)))

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = tools.loadCrossTrialRuleEncoding(subj,rule='Logic')
    scount += 1

## 1.1 Run across subject decoding on Logic rule encoding

In [36]:
nproc = 20
ncvs = 1

distances_baseline = ruleDecoding(data_task, rois=np.arange(nParcels), ncvs=ncvs, nproc=nproc)
    

## 1.2 Compute statistics

In [37]:
rois = np.arange(nParcels)
nStims = 4
statistics_logic = np.zeros((len(rois),3)) # acc, q, acc_thresh
for roicount in range(len(rois)):
    ntrials = len(subjNums)*nStims
    p = stats.binom_test(np.mean(distances_baseline[roicount,:])*ntrials,n=ntrials,p=1/float(nStims))
    if np.mean(distances_baseline[roicount,:])>1/float(nStims):
        p = p/2.0
    else:
        p = 1.0-p/2.0
        

    statistics_logic[roicount,0] = np.mean(distances_baseline[roicount,:])
    statistics_logic[roicount,1] = p

h0, qs = mc.fdrcorrection0(statistics_logic[:,1])
print 'Number of significant decodings:', np.sum(h0)
for roicount in range(len(rois)):
    statistics_logic[roicount,1] = qs[roicount]
    statistics_logic[roicount,2] = h0[roicount]*statistics_logic[roicount,0]
    if qs[roicount]<0.05:
        print 'ROI', roicount+1, 
        print '\tacc =', statistics_logic[roicount,0]
        print '\tp =', statistics_logic[roicount,1]
    

        
decodingROI = np.zeros((glasser2.shape[0],2)) # unthresh acc, thresh acc
sig_acc = np.multiply(statistics_logic[:,0],statistics_logic[:,1]<0.05)
for roi in range(nParcels):
    roi_ind = np.where(glasser2==roi+1)[0]

    decodingROI[roi_ind,0] = statistics_logic[roi,0]
    decodingROI[roi_ind,1] = sig_acc[roi]

# Compute effect size baseline (information content)
outdir = '/projects3/SRActFlow/data/results/Pilot4a_GroupRuleDecoding/'
filename = 'LogicRuleDecoding_64k'
np.savetxt(outdir + filename + '.csv', decodingROI,fmt='%s')
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + filename + '.dscalar.nii -reset-scalars'
os.system(wb_command)


Number of significant decodings: 250
ROI 2 	acc = 0.3671875
	p = 5.91977039345e-07
ROI 4 	acc = 0.3046875
	p = 0.0126209190213
ROI 5 	acc = 0.294270833333
	p = 0.0380537616086
ROI 6 	acc = 0.333333333333
	p = 0.000280181257733
ROI 7 	acc = 0.325520833333
	p = 0.000962606106077
ROI 9 	acc = 0.421875
	p = 9.55878164742e-13
ROI 10 	acc = 0.299479166667
	p = 0.0222489319672
ROI 11 	acc = 0.411458333333
	p = 1.63938732554e-11
ROI 12 	acc = 0.434895833333
	p = 3.63950852073e-14
ROI 13 	acc = 0.427083333333
	p = 2.67009619532e-13
ROI 15 	acc = 0.354166666667
	p = 7.53440604858e-06
ROI 16 	acc = 0.432291666667
	p = 5.8759772172e-14
ROI 17 	acc = 0.346354166667
	p = 3.44510189795e-05
ROI 18 	acc = 0.375
	p = 1.21191190918e-07
ROI 22 	acc = 0.315104166667
	p = 0.00348506532952
ROI 23 	acc = 0.315104166667
	p = 0.00348506532952
ROI 26 	acc = 0.348958333333
	p = 2.10673011697e-05
ROI 27 	acc = 0.447916666667
	p = 7.3999367955e-16
ROI 28 	acc = 0.385416666667
	p = 1.29232638207e-08
ROI 30 	acc = 0.

0

## 1.01 Load data for SENSORY rules

In [38]:
tools = reload(tools)

In [39]:
# gsr = True
n_rules = 4
data_task = np.zeros((len(glasser2),n_rules,len(subjNums)))

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = tools.loadCrossTrialRuleEncoding(subj,rule='Sensory')
    scount += 1

## 1.1 Run across subject decoding on Sensory rule encoding

In [40]:
nproc = 20
ncvs = 1

distances_baseline = ruleDecoding(data_task, rois=np.arange(nParcels), ncvs=ncvs, nproc=nproc)
    

## 1.2 Compute statistics

In [41]:
rois = np.arange(nParcels)
nStims = 4
statistics_sensory = np.zeros((len(rois),3)) # acc, q, acc_thresh
for roicount in range(len(rois)):
    ntrials = len(subjNums)*nStims
    p = stats.binom_test(np.mean(distances_baseline[roicount,:])*ntrials,n=ntrials,p=1/float(nStims))
    if np.mean(distances_baseline[roicount,:])>1/float(nStims):
        p = p/2.0
    else:
        p = 1.0-p/2.0
        

    statistics_sensory[roicount,0] = np.mean(distances_baseline[roicount,:])
    statistics_sensory[roicount,1] = p

h0, qs = mc.fdrcorrection0(statistics_sensory[:,1])
print 'Number of significant decodings:', np.sum(h0)
for roicount in range(len(rois)):
    statistics_sensory[roicount,1] = qs[roicount]
    statistics_sensory[roicount,2] = h0[roicount]*statistics_sensory[roicount,0]
    if qs[roicount]<0.05:
        print 'ROI', roicount+1, 
        print '\tacc =', statistics_sensory[roicount,0]
        print '\tp =', statistics_sensory[roicount,1]
    

        
decodingROI = np.zeros((glasser2.shape[0],2)) # unthresh acc, thresh acc
sig_acc = np.multiply(statistics_sensory[:,0],statistics_sensory[:,1]<0.05)
for roi in range(nParcels):
    roi_ind = np.where(glasser2==roi+1)[0]

    decodingROI[roi_ind,0] = statistics_sensory[roi,0]
    decodingROI[roi_ind,1] = sig_acc[roi]

# Compute effect size baseline (information content)
outdir = '/projects3/SRActFlow/data/results/Pilot4a_GroupRuleDecoding/'
filename = 'SensoryRuleDecoding_64k'
np.savetxt(outdir + filename + '.csv', decodingROI,fmt='%s')
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + filename + '.dscalar.nii -reset-scalars'
os.system(wb_command)


Number of significant decodings: 180
ROI 2 	acc = 0.359375
	p = 8.83772021039e-06
ROI 5 	acc = 0.4375
	p = 3.8831131502e-14
ROI 6 	acc = 0.401041666667
	p = 8.39907895453e-10
ROI 7 	acc = 0.403645833333
	p = 4.31185936306e-10
ROI 8 	acc = 0.302083333333
	p = 0.0232005316744
ROI 9 	acc = 0.3125
	p = 0.00736437949806
ROI 11 	acc = 0.424479166667
	p = 1.22589759384e-12
ROI 12 	acc = 0.356770833333
	p = 1.41794556296e-05
ROI 13 	acc = 0.3359375
	p = 0.000347543977845
ROI 14 	acc = 0.354166666667
	p = 1.77698255863e-05
ROI 15 	acc = 0.3203125
	p = 0.00327048986034
ROI 16 	acc = 0.322916666667
	p = 0.00232271817502
ROI 17 	acc = 0.395833333333
	p = 3.78856296913e-09
ROI 18 	acc = 0.4140625
	p = 2.68640960168e-11
ROI 19 	acc = 0.346354166667
	p = 7.09588152115e-05
ROI 20 	acc = 0.359375
	p = 8.83772021039e-06
ROI 21 	acc = 0.328125
	p = 0.00111926144004
ROI 23 	acc = 0.299479166667
	p = 0.0303982500715
ROI 26 	acc = 0.361979166667
	p = 5.38224012567e-06
ROI 27 	acc = 0.354166666667
	p = 1.776

0

## 1.01 Load data for MOTOR rules

In [42]:
tools = reload(tools)

In [43]:
# gsr = True
n_rules = 4
data_task = np.zeros((len(glasser2),n_rules,len(subjNums)))

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = tools.loadCrossTrialRuleEncoding(subj,rule='Motor')
    scount += 1

## 1.1 Run across subject decoding on Motor rule encoding

In [44]:
nproc = 20
ncvs = 1

distances_baseline = ruleDecoding(data_task, rois=np.arange(nParcels), ncvs=ncvs, nproc=nproc)
    

## 1.2 Compute statistics

In [45]:
rois = np.arange(nParcels)
nStims = 4
statistics_motor = np.zeros((len(rois),3)) # acc, q, acc_thresh
for roicount in range(len(rois)):
    ntrials = len(subjNums)*nStims
    p = stats.binom_test(np.mean(distances_baseline[roicount,:])*ntrials,n=ntrials,p=1/float(nStims))
    if np.mean(distances_baseline[roicount,:])>1/float(nStims):
        p = p/2.0
    else:
        p = 1.0-p/2.0
        

    statistics_motor[roicount,0] = np.mean(distances_baseline[roicount,:])
    statistics_motor[roicount,1] = p

h0, qs = mc.fdrcorrection0(statistics_motor[:,1])
print 'Number of significant decodings:', np.sum(h0)
for roicount in range(len(rois)):
    statistics_motor[roicount,1] = qs[roicount]
    statistics_motor[roicount,2] = h0[roicount]*statistics_motor[roicount,0]
    if qs[roicount]<0.05:
#         print 'ROI', roicount+1, 
#         print '\tacc =', statistics_motor[roicount,0]
#         print '\tp =', statistics_motor[roicount,1]
    

        
decodingROI = np.zeros((glasser2.shape[0],2)) # unthresh acc, thresh acc
sig_acc = np.multiply(statistics_motor[:,0],statistics_motor[:,1]<0.05)
for roi in range(nParcels):
    roi_ind = np.where(glasser2==roi+1)[0]

    decodingROI[roi_ind,0] = statistics_motor[roi,0]
    decodingROI[roi_ind,1] = sig_acc[roi]

# Compute effect size baseline (information content)
outdir = '/projects3/SRActFlow/data/results/Pilot4a_GroupRuleDecoding/'
filename = 'MotorRuleDecoding_64k'
np.savetxt(outdir + filename + '.csv', decodingROI,fmt='%s')
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + filename + '.dscalar.nii -reset-scalars'
os.system(wb_command)


Number of significant decodings: 50
ROI 9 	acc = 0.484375
	p = 9.56081653256e-21
ROI 10 	acc = 0.481770833333
	p = 1.66451095723e-20
ROI 32 	acc = 0.322916666667
	p = 0.00751467644859
ROI 41 	acc = 0.447916666667
	p = 7.3999367955e-16
ROI 42 	acc = 0.333333333333
	p = 0.00166033337916
ROI 45 	acc = 0.325520833333
	p = 0.00543406672786
ROI 52 	acc = 0.4765625
	p = 7.36445974917e-20
ROI 53 	acc = 0.450520833333
	p = 3.02768587454e-16
ROI 54 	acc = 0.4140625
	p = 2.91027706849e-11
ROI 55 	acc = 0.473958333333
	p = 1.873058756e-19
ROI 56 	acc = 0.3515625
	p = 7.10056475326e-05
ROI 71 	acc = 0.307291666667
	p = 0.0402644858814
ROI 75 	acc = 0.317708333333
	p = 0.014144993634
ROI 97 	acc = 0.377604166667
	p = 3.54773614482e-07
ROI 101 	acc = 0.328125
	p = 0.00378233176288
ROI 102 	acc = 0.364583333333
	p = 6.04055255021e-06
ROI 103 	acc = 0.325520833333
	p = 0.00543406672786
ROI 105 	acc = 0.3203125
	p = 0.0103418192881
ROI 107 	acc = 0.315104166667
	p = 0.0161285581529
ROI 114 	acc = 0.3098

0

# Evaluate Overlap of accuracy/representations 

In [50]:
rois = np.arange(nParcels)
nStims = 4

statistics_all = np.multiply(statistics_logic[:,1]<0.05,statistics_sensory[:,1]<0.05)
statistics_all = np.multiply(statistics_all,statistics_motor[:,1]<0.05)

print 'Number of signifiicant overlaps:', np.sum(statistics_all)
        
decodingROI = np.zeros((glasser2.shape[0],4)) # unthresh acc, thresh acc
sig_acc = np.multiply(statistics_motor[:,0],statistics_motor[:,1]<0.05)
for roi in range(nParcels):
    roi_ind = np.where(glasser2==roi+1)[0]

    decodingROI[roi_ind,0] = statistics_all[roi]
    decodingROI[roi_ind,1] = np.multiply(statistics_all[roi],statistics_logic[roi,0])
    decodingROI[roi_ind,2] = np.multiply(statistics_all[roi],statistics_sensory[roi,0])
    decodingROI[roi_ind,3] = np.multiply(statistics_all[roi],statistics_motor[roi,0])

# Compute effect size baseline (information content)
outdir = '/projects3/SRActFlow/data/results/Pilot4a_GroupRuleDecoding/'
filename = 'OverlapRuleDecoding_64k'
np.savetxt(outdir + filename + '.csv', decodingROI,fmt='%s')
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + filename + '.dscalar.nii -reset-scalars'
os.system(wb_command)


Number of signifiicant overlaps: 21


0